In [ ]:
import os
from ts_data.ts_data import ts_data as ts
import visualize
import ts_models.predicts
import pandas as pd
import ts_models.ts_lstm as ts_lstm

pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

In [ ]:
# view tenslow enabled devices
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = '1' # -1 to disable GPU

# Load Prepped Data Object

In [ ]:
import pickle
ts_data = pickle.load(open('ts_data/ts_data.pkl','rb'))

# Build LSTM Architecture

In [ ]:
ts_data.train_X.shape[-1]

In [ ]:
ts_model = ts_lstm.lstm_model(
    ts_data,
    inlayer=int(ts_data.train_X.shape[-1])*2,
    hiddenlayers=[25],
    loss_function='mae',
    dropout=0.05,
    activation='tanh',
    gpus=1)

# Fit model

In [ ]:
import time
start = time.time()

history = ts_model.fit(
    ts_data.train_X, ts_data.train_y,
    epochs=55, 
    batch_size=256, 
    validation_data=(ts_data.test_X, ts_data.test_y), 
    verbose=2, 
    shuffle=False)

fitTime = time.time()-start
print('Fit Time: {}'.format(round(fitTime,2)))

In [ ]:
ts_model.save('amd_fit.h5')

In [ ]:
visualize.plot_loss(history)

# Results

In [ ]:
# select the last sequence of data (it is length of n_in)
test_X = ts_data.test_X[-1]
test_Y = ts_data.test_y[-1]

In [ ]:
ts_data.test_X.shape

In [ ]:
test_X.reshape(1, test_X.shape[-2], test_X.shape[-1]).shape

In [ ]:
yhat = ts_model.predict(test_X.reshape(1, test_X.shape[-2], test_X.shape[-1]))

In [ ]:
import matplotlib.pyplot as plt
x = [x for x in range(len(yhat[0]))]
plt.plot(x, yhat[0], label='predicted')
plt.plot(x, test_Y, label='actual')
plt.title(ts_data.entityID)
plt.xlabel('days')
plt.ylabel('amount change in price')
plt.legend()
plt.show()